<a href="https://colab.research.google.com/github/Yunho-seo/education_center6/blob/main/01_%EB%8C%80%EC%A0%84_%EC%8B%A4%EC%8A%B5(front).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 수신부

In [ ]:
!pip install paho_mqtt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.4/99.4 kB 5.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for paho_mqtt: filename=paho_mqtt-1.6.1-py3-none-any.whl size=62118 sha256=8d4a1d4a2a94952361a79566982c29cbd13ad0d2ce114a3c7c765f8c8abaad53
  Stored in directory: /root/.cache/pip/wheels/8b/bb/0c/79444d1dee20324d442856979b5b519b48828b0bd3d05df84a
Successfully built paho_mqtt


In [ ]:
import paho.mqtt.client as mqtt
client = mqtt.Client()  # MQTT 클라이언트 객체 생성
# client.username_pw_set(username = 'testamin', password = '')
client.connect('34.64.86.80', 19883)  # MQTT 브로커에 연결, IP 주소와 포트번호 지정
client.publish('house/light', '빨리켜지기')  # housse/light 토픽으로 '빨리켜지기'라는 메시지를 게시

In [ ]:
client.subscribe('house/light', qos = 0)

(0, 2)

In [ ]:
def on_connect(client, userdata, flags, rc):
  print('연결되었습니다.' + str(rc))
def on_message(client, userdata, msg):  # 함수 형태가 정해져 있음
  print(msg.topic + '' + str(msg.payload))

In [ ]:
# 수신 프로그램

In [ ]:
# 이벤트 처리
client = mqtt.Client()
# client.username_pw_set(username = 'testmin', password = '')
# on_ : 웹에서 event 처리할 때 사용
client.on_connect = on_connect
client.on_message = on_message
client.connect('34.64.86.80', 19883)
client.subscribe('house/light', qos = 1)  # 토픽 (topic, root로부터 시작하지 않음)
                                          # 토픽은 MQTT에서 메시지를 게시하거나 구독하는 데에 사용되는 주제를 나타내는 문자열
client.loop_forever()

In [ ]:
# JSON 수신 프로그램

In [ ]:
import json
def on_connect(client, userdata, flags, rc):
  print("연결되었습니다 " + str(rc))
def on_message(client, userdata, msg):
  json_return = json.loads(msg.payload.decode('utf-8'))
  print(json_return["light"])
  print(msg.topic + " " + str(json_return))

In [ ]:
client = mqtt.Client()
client.username_pw_set(username = "daejeon", password = "daejeon1234")
client.on_connect = on_connect
client.on_message = on_message
client.connect('34.64.86.80', 19883)
client.subscribe('house/light', qos = 1)
client.loop_forever

In [ ]:
!pip install pymysql

In [ ]:
import json
import pymysql

con = pymysql.connect(host='34.64.86.80', port = 3306, user = 'root', passwd = 'acorn1234',
                     use_unicode = True, db = 'mqtt', charset = 'utf8', autocommit=True)
cursor = con.cursor()

def on_connect(client, userdata, flags, rc):
  print("연결되었습니다 " + str(rc))
  client.subscribe('house/light')

def on_message(client, userdata, msg):
  json_return = json.loads(msg.payload.decode('utf-8'))  # 한글 표현
  print(json_return["light"])
  print(json_return["hangeul"])
  print(msg.topic + " " + str(json_return))
  try:
    with con.cursor() as cursor:
      sql = "insert into mqtt (light, door, close, hangeul) values (%s, %s, %s, %s)"
      cursor.execute(sql, (json_return['light'], json_return['door'], json_return['close'], json_return['hangeul']))
    con.commit()
    print('저장 완료')
  except Exception as e:
    print('실행 오류:', e)
  if json_return['close'] == 'true':
    client.disconnect()

def on_disconnect(client, userdata, rc):
  print("해제 이유: " + str(rc))
  con.close()
  client.connected_flag = False
  client.disconnect_flag = True

def on_subscribed(client, userdata, topic):
  print('다음 토픽에 등록됨 : $topic')

def on_subscribe(client, userdata, mid, granted_qos):
  print("subscribed: " + str(mid) + " " + str(granted_qos))

def on_close(client, userdata, rc):
  print("종료")
  con.close()

In [ ]:
client = mqtt.Client()
client.username_pw_set(username = 'mirae', password='daejeon1234')
client.on_connect = on_connect
client.on_message = on_message
client.on_disconnect = on_disconnect
client.on_subscribe = on_subscribe
client.on_subscribed = on_subscribed
client.on_close = on_close
client.connect('34.64.86.80', 19883)
client.subscribe('house/light', qos=1)
client.loop_forever()

연결되었습니다 0
subscribed: 1 (1,)
subscribed: 2 (0,)
on
대전
house/light {'light': 'on', 'door': 'close', 'close': 'false', 'hangeul': '대전'}
저장 완료
on
대전
house/light {'light': 'on', 'door': 'close', 'close': 'true', 'hangeul': '대전'}
저장 완료
해제 이유: 0


7